# Engineering daily features of folad stocks

In [17]:
!pip install jalali-pandas 

In [18]:
import numpy as np
import pandas as pd

import pytse_client as tse
from pytse_client.download import download_financial_indexes

import jalali_pandas

import matplotlib.pyplot as plt

In [2]:
marketIndex = download_financial_indexes(symbols = 'شاخص كل', write_to_csv = True, include_jdate = True)
marketIndex = marketIndex['شاخص كل']
marketIndex.rename(columns = {'value': 'TEDPIX'}, inplace = True)
print(f'Shape of TEDPIX: {marketIndex.shape}')
marketIndex.head()

Shape of TEDPIX: (3328, 3)


,jdate,TEDPIX,date
0,1387-09-14,9248.9,2008-12-04
1,1387-09-15,9248.9,2008-12-05
2,1387-09-16,9178.3,2008-12-06
3,1387-09-17,9130.5,2008-12-07
4,1387-09-18,9089.2,2008-12-08


In [3]:
tse.download(symbols = ['فولاد'], write_to_csv = True, include_jdate = True)
folad = tse.Ticker(symbol = 'فولاد',  adjust = True)
foladHist = folad.history
foladHist['pct_change'] = foladHist['adjClose'].pct_change() * 100
print(f'Shape of Folad price history: {foladHist.shape}')

foladHist.head()

Shape of Folad price history: (3417, 11)


,date,open,high,low,adjClose,value,volume,count,yesterday,close,pct_change
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0,NaN
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0,0.0
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0,0.0
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0,0.0
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0,0.0


In [4]:
foladClient = folad.client_types
print(f"Shape of Folad's client history: {foladClient.shape}")
foladClient.to_csv('foladClient.csv')
foladClient.head()

Shape of Folad's client history: (3030, 18)


,date,individual_buy_count,corporate_buy_count,individual_sell_count,corporate_sell_count,individual_buy_vol,corporate_buy_vol,individual_sell_vol,corporate_sell_vol,individual_buy_value,corporate_buy_value,individual_sell_value,corporate_sell_value,individual_buy_mean_price,individual_sell_mean_price,corporate_buy_mean_price,corporate_sell_mean_price,individual_ownership_change
0,20220913,2337,8,958,20,55162895,22115512,28376476,48901931,285905053830,114743237140,147157944330,253490346640,5182.923301,5185.913301,5188.359968,5183.646974,26786419.0
1,20220912,3240,7,899,17,45888066,8109930,21720353,32277643,234063072560,41313078170,110825079310,164551071420,5100.739538,5102.360874,5094.134989,5097.989076,24167713.0
2,20220911,1371,7,867,13,27460629,10290000,20446098,17304531,138161463190,51693700000,102801001160,87054162030,5031.256319,5027.903180,5023.683188,5030.714905,7014531.0
3,20220910,1456,8,1076,13,29062906,21378562,27200089,23241379,146260987130,107612798860,137059128840,116814657150,5032.565812,5038.922073,5033.678077,5026.150004,1862817.0
4,20220907,1449,6,857,14,32216792,10367000,18179929,24403863,162903228610,52387990720,91866598120,123424621210,5056.469577,5053.187948,5053.341441,5057.585400,14036863.0


In [5]:
foladHist.insert(1,'step', np.arange(1, len(foladHist) + 1))
foladHist = pd.merge(foladHist, marketIndex, how = 'left', on = 'date')

cols = ['date', 'jdate', 'step', 'TEDPIX',
         'open', 'close', 'high', 'low', 'adjClose', 
        'value', 'volume', 'count', 'yesterday']
foladHist = foladHist[cols]

foladHist.head()

,date,jdate,step,TEDPIX,open,close,high,low,adjClose,value,volume,count,yesterday
0,2007-03-11,NaN,1,NaN,16.0,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0
1,2007-03-12,NaN,2,NaN,16.0,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0
2,2007-03-13,NaN,3,NaN,16.0,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0
3,2007-03-14,NaN,4,NaN,16.0,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0
4,2007-03-17,NaN,5,NaN,16.0,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0


In [6]:
foladClient['date'] = pd.to_datetime(foladClient['date'])
foladClient.iloc[:, 1: 13] = foladClient.iloc[:, 1: 13].astype('float')

foladClient['Enter_individual_money'] = foladClient['individual_buy_value'] - foladClient['individual_sell_value']
foladClient['Individual_buy_per_capita'] = foladClient['individual_buy_value'] / foladClient['individual_buy_count']
foladClient['Individual_sell_per_capita'] = foladClient['individual_sell_value'] / foladClient['individual_sell_count']
foladClient['Individual_buy_power_to_Individual_sell'] = foladClient['Individual_buy_per_capita'] / foladClient['Individual_sell_per_capita']

foladClient.head()

,date,individual_buy_count,corporate_buy_count,individual_sell_count,corporate_sell_count,individual_buy_vol,corporate_buy_vol,individual_sell_vol,corporate_sell_vol,individual_buy_value,...,corporate_sell_value,individual_buy_mean_price,individual_sell_mean_price,corporate_buy_mean_price,corporate_sell_mean_price,individual_ownership_change,Enter_individual_money,Individual_buy_per_capita,Individual_sell_per_capita,Individual_buy_power_to_Individual_sell
0,2022-09-13,2337.0,8.0,958.0,20.0,55162895.0,22115512.0,28376476.0,48901931.0,2.859051e+11,...,2.534903e+11,5182.923301,5185.913301,5188.359968,5183.646974,26786419.0,1.387471e+11,1.223385e+08,1.536095e+08,0.796425
1,2022-09-12,3240.0,7.0,899.0,17.0,45888066.0,8109930.0,21720353.0,32277643.0,2.340631e+11,...,1.645511e+11,5100.739538,5102.360874,5094.134989,5097.989076,24167713.0,1.232380e+11,7.224169e+07,1.232760e+08,0.586016
2,2022-09-11,1371.0,7.0,867.0,13.0,27460629.0,10290000.0,20446098.0,17304531.0,1.381615e+11,...,8.705416e+10,5031.256319,5027.903180,5023.683188,5030.714905,7014531.0,3.536046e+10,1.007742e+08,1.185709e+08,0.849907
3,2022-09-10,1456.0,8.0,1076.0,13.0,29062906.0,21378562.0,27200089.0,23241379.0,1.462610e+11,...,1.168147e+11,5032.565812,5038.922073,5033.678077,5026.150004,1862817.0,9.201858e+09,1.004540e+08,1.273784e+08,0.788627
4,2022-09-07,1449.0,6.0,857.0,14.0,32216792.0,10367000.0,18179929.0,24403863.0,1.629032e+11,...,1.234246e+11,5056.469577,5053.187948,5053.341441,5057.585400,14036863.0,7.103663e+10,1.124246e+08,1.071956e+08,1.048780


In [7]:
foladMerge = pd.merge(foladHist, foladClient, how = 'left', on = 'date')
foladMerge.columns

Index(['date', 'jdate', 'step', 'TEDPIX', 'open', 'close', 'high', 'low',
       'adjClose', 'value', 'volume', 'count', 'yesterday',
       'individual_buy_count', 'corporate_buy_count', 'individual_sell_count',
       'corporate_sell_count', 'individual_buy_vol', 'corporate_buy_vol',
       'individual_sell_vol', 'corporate_sell_vol', 'individual_buy_value',
       'corporate_buy_value', 'individual_sell_value', 'corporate_sell_value',
       'individual_buy_mean_price', 'individual_sell_mean_price',
       'corporate_buy_mean_price', 'corporate_sell_mean_price',
       'individual_ownership_change', 'Enter_individual_money',
       'Individual_buy_per_capita', 'Individual_sell_per_capita',
       'Individual_buy_power_to_Individual_sell'],
      dtype='object')

In [22]:
# Log is for better mapping to target prediction

def ret_based_func(foladHist):
    df = foladHist
    return_df = pd.DataFrame({\
                              'ret1d_log':(np.log(df.adjClose) - np.log(df.yesterday)), \
                              'ret3d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(3).sum(), \
                              'ret7d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(7).sum(), \
                              'ret14d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(14).sum(), \
                              'ret30d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(30).sum(), \
                              'ret60d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(60).sum(), \
                              'ret90d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(90).sum(), \
                              'ret120d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(120).sum(), \
                              'ret300d_log':(np.log(df.adjClose) - np.log(df.yesterday)).rolling(300).sum(), \
                              'lastclose_log':(np.log(df['adjClose']) - np.log(df['close'])), \
                              'buy_queue_locked' : np.logical_and(np.isclose(df['high'], df['low']), df['high'] > df['yesterday']), \
                              'sell_queue_locked' : np.logical_and(np.isclose(df['high'], df['low']), df['low'] < df['yesterday']), \
                              })
    return return_df

price_features = ret_based_func(foladMerge)
price_features

,ret1d_log,ret3d_log,ret7d_log,ret14d_log,ret30d_log,ret60d_log,ret90d_log,ret120d_log,ret300d_log,lastclose_log,buy_queue_locked,sell_queue_locked
2007-03-11,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,False,False
2007-03-12,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,False,False
2007-03-13,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,False,False
2007-03-14,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,False,False
2007-03-17,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-10,-0.005946,-0.009891,-0.027453,-0.074676,-0.053418,-0.109309,-0.110912,0.021908,0.268646,-0.001986,False,False
2022-09-11,0.000000,-0.005946,-0.021633,-0.054172,-0.032854,-0.089873,-0.130819,0.021908,0.298476,0.000000,False,False
2022-09-12,0.013821,0.007874,0.001963,-0.032790,-0.010921,-0.077687,-0.120133,0.035729,0.274953,-0.005865,False,False
2022-09-13,0.015565,0.029385,0.017527,-0.028546,0.010284,-0.062122,-0.056118,0.063767,0.238708,-0.007692,False,False


In [23]:
def prp_based_func(foladHist):
    df = foladHist
    df.index = [pd.Timestamp(item) for item in df['date']]
    return_df = pd.DataFrame({\
        'prp_high30d':df['adjClose']/df['high'].rolling('30d').max(), \
        'prp_high60d':df['adjClose']/df['high'].rolling('60d').max(), \
        'prp_high90d':df['adjClose']/df['high'].rolling('90d').max(), \
        'prp_high120d':df['adjClose']/df['high'].rolling('120d').max(), \
        'prp_high300d':df['adjClose']/df['high'].rolling('300d').max(), \
        'prp_low30d':df['adjClose']/df['low'].rolling('30d').min(), \
        'prp_low60d':df['adjClose']/df['low'].rolling('60d').min(), \
        'prp_low90d':df['adjClose']/df['low'].rolling('90d').min(), \
        'prp_low120d':df['adjClose']/df['low'].rolling('120d').min(), \
        'prp_low300d':df['adjClose']/df['low'].rolling('300d').min(), \
        'prp_value3d30d':df['value'].rolling('3d').mean()/df['value'].rolling('30d').mean(), \
        'prp_value5d60d':df['value'].rolling('5d').mean()/df['value'].rolling('60d').mean(), \
        'prp_value15d120d':df['value'].rolling('15d').mean()/df['value'].rolling('120d').mean(), \
        'prp_value30d200d':df['value'].rolling('30d').mean()/df['value'].rolling('200d').mean(), \
        })
    return return_df

prop_features = prp_based_func(foladMerge)
prop_features

,prp_high30d,prp_high60d,prp_high90d,prp_high120d,prp_high300d,prp_low30d,prp_low60d,prp_low90d,prp_low120d,prp_low300d,prp_value3d30d,prp_value5d60d,prp_value15d120d,prp_value30d200d
2007-03-11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2007-03-12,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2007-03-13,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2007-03-14,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.476596,1.000000,1.000000,1.000000
2007-03-17,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.099964,0.388384,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-10,0.859829,0.859829,0.859829,0.831817,0.831817,1.008016,1.021112,1.021112,1.021112,1.178262,0.761712,0.486124,0.376277,0.470836
2022-09-11,0.859829,0.859829,0.859829,0.831817,0.831817,1.008016,1.021112,1.021112,1.021112,1.178262,0.679600,0.508841,0.370173,0.462695
2022-09-12,0.871795,0.871795,0.871795,0.843393,0.843393,1.022044,1.035323,1.035323,1.035323,1.194659,0.803278,0.561589,0.375486,0.425143
2022-09-13,0.900870,0.885470,0.885470,0.856623,0.856623,1.038076,1.051563,1.051563,1.051563,1.213399,0.960099,0.657079,0.407252,0.429865


In [26]:
def wght_based_feature(foladHist):
    print('weight based features')
    
    def value20d_based_func(foladHist):
        return_df = pd.DataFrame({\
            'value_20d':foladHist['value'].rolling(20).mean(), \
            })
        return return_df
    
    df_value20_data = pd.concat([foladHist[['date','value']], value20d_based_func(foladHist)], axis=1)
    
    def wght_based_func(stock_date_group):
        return_df = pd.DataFrame({\
            'value_weight':foladHist['value']/foladHist['value'].sum(), \
            'value_weight20d':foladHist['value_20d']/foladHist['value_20d'].sum(), \
            })
        return return_df

    df_wght_based = value20d_based_func(foladHist)


    df_feature = pd.concat([df_value20_data, df_wght_based], axis=1)
    return df_feature

wght_data = wght_based_feature(foladMerge)
wght_data

weight based features


,date,value,value_20d,value_20d
2007-03-11,2007-03-11,889437216900,NaN,NaN
2007-03-12,2007-03-12,193879458000,NaN,NaN
2007-03-13,2007-03-13,249241504527,NaN,NaN
2007-03-14,2007-03-14,51666379451,NaN,NaN
2007-03-17,2007-03-17,28239006789,NaN,NaN
...,...,...,...,...
2022-09-10,2022-09-10,253873785990,3.332936e+11,3.332936e+11
2022-09-11,2022-09-11,189855163190,2.995558e+11,2.995558e+11
2022-09-12,2022-09-12,275376150730,2.956202e+11,2.956202e+11
2022-09-13,2022-09-13,400648290970,2.958984e+11,2.958984e+11


In [41]:
def indv_nonindv_based_feature(foladMerge, wght_data, prp_data):
    print('individual and non-individual features')
    features = pd.DataFrame([])

    # mapping
    def pcap_based_func(foladMerge):
        df = foladMerge
        return_df = pd.DataFrame({\
            'Individual_buy_per_capita':np.where(df['individual_buy_count'] <= 0, \
                0.0, df['individual_buy_value']/df['individual_buy_count']), \
            'Individual_sell_per_capita':np.where(df['individual_sell_count'] <= 0, \
                0.0, df['individual_sell_value']/df['individual_sell_count']), \
            'indv_net_count_prp7d30':(df['individual_buy_count']-df['individual_sell_count']).rolling(7).sum()/\
                (df['individual_buy_count']+df['individual_sell_count']).rolling(30).sum(), \
            'indv_net_count_prp7d120':(df['individual_buy_count']-df['individual_sell_count']).rolling(7).sum()/\
                (df['individual_buy_count']+df['individual_sell_count']).rolling(120).sum(), \
            'indv_net_value':(df['individual_buy_value']-df['individual_sell_value']), \
            'indv_relnet_value_d14':((df['individual_buy_value']-df['individual_sell_value'])/\
                (df['individual_buy_value']+df['individual_sell_value'])).rolling(14).sum(), \
            'indv_relnet_value_d30':((df['individual_buy_value']-df['individual_sell_value'])/\
                (df['individual_buy_value']+df['individual_sell_value'])).rolling(30).sum(), \
            'indv_relnet_value_d60':((df['individual_buy_value']-df['individual_sell_value'])/\
                (df['individual_buy_value']+df['individual_sell_value'])).rolling(60).sum(), \
            'indv_relnet_value_d90':((df['individual_buy_value']-df['individual_sell_value'])/\
                (df['individual_buy_value']+df['individual_sell_value'])).rolling(90).sum(), \
            'indv_relnet_value_d120':((df['individual_buy_value']-df['individual_sell_value'])/\
                (df['individual_buy_value']+df['individual_sell_value'])).rolling(120).sum(), \
            'indv_relnet_value_d300':((df['individual_buy_value']-df['individual_sell_value'])/\
                (df['individual_buy_value']+df['individual_sell_value'])).rolling(300).sum(), \
            'indv_relnet5d10d_value_d10':((df['individual_buy_value'].rolling(5).mean()-df['individual_sell_value'].rolling(10).mean())/\
                (df['individual_buy_value']+df['individual_sell_value'])).rolling(10).mean(), \
            })
        return return_df
    
    pcap_feature_data = pcap_based_func(foladMerge)

    # folad percapita dataframe 
    stock_percapita = pd.concat([foladMerge[[ 'date', 'individual_buy_value', 'individual_sell_value']], \
        pcap_feature_data, wght_data, prp_data], axis=1)
    
    # mapping
    def mrkt_pcap_based_func(stock_percapita):
        initial_index = stock_percapita.index
        df = stock_percapita
        one_vec = np.ones([len(df)])
        return_df = pd.DataFrame({\
            'mrkt_indv_buy_pcap':(df['Individual_buy_per_capita']*df['value_weight20d']).sum()*one_vec, \
            'mrkt_indv_sell_pcap':(df['Individual_sell_per_capita']*df['value_weight20d']).sum()*one_vec, \
            'mrkt_indv_net_value':((df['individual_buy_value'] - df['Individual_sell_value'])*df['value_weight20d']).sum()*one_vec, \
            'mrkt_indv_absnet_value':(np.abs(df['individual_buy_value'] - df['individual_sell_value'])*df['value_weight20d']).sum()*one_vec, \
            'mrkt_prp_high30d':df['prp_high30d'].mean()*one_vec,\
            'mrkt_prp_high60d':df['prp_high60d'].mean()*one_vec,\
            'mrkt_prp_high90d':df['prp_high90d'].mean()*one_vec,\
            'mrkt_prp_high120d':df['prp_high120d'].mean()*one_vec,\
            'mrkt_prp_high300d':df['prp_high300d'].mean()*one_vec,\
            'mrkt_prp_low30d':df['prp_low30d'].mean()*one_vec,\
            'mrkt_prp_low60d':df['prp_low60d'].mean()*one_vec,\
            'mrkt_prp_low90d':df['prp_low90d'].mean()*one_vec,\
            'mrkt_prp_low120d':df['prp_low120d'].mean()*one_vec,\
            'mrkt_prp_low300d':df['prp_low300d'].mean()*one_vec\
            })
        return_df.index = initial_index
        return return_df
    
    mrkt_pcap_feature_data = pcap_based_func(foladMerge)
    
    # folad percapita dataframe 
    stock_percapita = pd.concat([stock_percapita, mrkt_pcap_feature_data], axis=1)

    # market rolling 
    def mrkt_rolling_based_func(stock_percapita):
        df = stock_percapita
        df.index = [pd.Timestamp(item) for item in df['date']]
        return_df = pd.DataFrame({\
        'mrkt_indv_absnet_value3d': df['mrkt_indv_absnet_value'].rolling(3).sum(),\
        })
        return return_df
    
    mrkt_rolling_data = pcap_based_func(foladMerge)

    prc_mrkt_data = pd.concat([stock_percapita, mrkt_rolling_data], axis=1)

    # mapping
    # TODO: for daily powers => make them bolder!/ for others: use mean instead of sum
    def relpcap_based_func(prc_mrkt_data):
        df = prc_mrkt_data
        df.index = [pd.Timestamp(item) for item in df['date']]
        return_df = pd.DataFrame({\
            'indv_power':((df['Individual_buy_per_capita'] - df['Individual_sell_per_capita'])\
                /(df['Individual_buy_per_capita'] + df['Individual_sell_per_capita'])), \
            'indv_power3d':(df['Individual_buy_per_capita'] - df['Individual_sell_per_capita']).rolling(3).sum()\
                /(df['Individual_buy_per_capita'] + df['Individual_sell_per_capita']).rolling(3).mean(), \
            'indv_power5d':(df['Individual_buy_per_capita'] - df['Individual_sell_per_capita']).rolling(5).sum()\
                /(df['Individual_buy_per_capita'] + df['Individual_sell_per_capita']).rolling(5).mean(), \
            'indv_power7d':(df['Individual_buy_per_capita'] - df['Individual_sell_per_capita']).rolling(7).sum()\
                /(df['Individual_buy_per_capita'] + df['Individual_sell_per_capita']).rolling(7).mean(), \
            'indv_power14d':(df['Individual_buy_per_capita'] - df['Individual_sell_per_capita']).rolling(14).sum()\
                /(df['Individual_buy_per_capita'] + df['Individual_sell_per_capita']).rolling(14).mean(), \
            'indv_power30d':(df['Individual_buy_per_capita'] - df['Individual_sell_per_capita']).rolling(30).sum()\
                /(df['Individual_buy_per_capita'] + df['Individual_sell_per_capita']).rolling(30).mean(), \
            'indv_power5d25':(df['Individual_buy_per_capita'] - df['Individual_sell_per_capita']).rolling(5).sum()\
                /(df['Individual_buy_per_capita'] - df['Individual_sell_per_capita']).rolling(25).sum(),\
            'indv_buy_pcap_prpd7d30':df['Individual_buy_per_capita'].rolling(7).mean()/df['Individual_buy_per_capita'].rolling(30).mean(), \
            'indv_buy_pcap_prpd14d60':df['Individual_buy_per_capita'].rolling(14).mean()/df['Individual_buy_per_capita'].rolling(60).mean(), \
            'indv_buy_pcap_prpd30d120':df['Individual_buy_per_capita'].rolling(30).mean()/df['Individual_buy_per_capita'].rolling(120).mean(), \
            'indv_sell_pcap_prpd7d30':df['Individual_buy_per_capita'].rolling(7).mean()/df['indv_sell_pcap'].rolling(30).mean(), \
            'indv_sell_pcap_prpd14d60':df['Individual_buy_per_capita'].rolling(14).mean()/df['indv_sell_pcap'].rolling(60).mean(), \
            'indv_sell_pcap_prpd30d120':df['Individual_sell_per_capita'].rolling(30).mean()/df['Individual_sell_per_capita'].rolling(120).mean(), \
            'indv_buy_pcap_prp_mrkt':df['Individual_buy_per_capita']/df['mrkt_indv_buy_pcap'], \
            'indv_sell_pcap_prp_mrkt':df['Individual_sell_per_capita']/df['mrkt_indv_sell_pcap'], \
            'indv_net_dev_mrkt':(df['indv_net_value'] - df['mrkt_indv_net_value'])/df['mrkt_indv_absnet_value'], \
            'indv_net_dev_mrkt3d':((df['indv_net_value'] - df['mrkt_indv_net_value']).rolling(3).sum())/df['mrkt_indv_absnet_value3d'], \
            'prp_high30d_dev_mrkt':df['prp_high30d'] - df['mrkt_prp_high30d'], \
            'prp_high60d_dev_mrkt':df['prp_high60d'] - df['mrkt_prp_high60d'], \
            'prp_high90d_dev_mrkt':df['prp_high90d'] - df['mrkt_prp_high90d'], \
            'prp_high120d_dev_mrkt':df['prp_high120d'] - df['mrkt_prp_high120d'], \
            'prp_high300d_dev_mrkt':df['prp_high300d'] - df['mrkt_prp_high300d'], \
            'prp_low30d_dev_mrkt':df['prp_low30d'] - df['mrkt_prp_low30d'], \
            'prp_low60d_dev_mrkt':df['prp_low60d'] - df['mrkt_prp_low60d'], \
            'prp_low90d_dev_mrkt':df['prp_low90d'] - df['mrkt_prp_low90d'], \
            'prp_low120d_dev_mrkt':df['prp_low120d'] - df['mrkt_prp_low120d'], \
            'prp_low300d_dev_mrkt':df['prp_low300d'] - df['mrkt_prp_low300d'], \
            })
        return return_df
    
    relpcap_feature_data = pcap_based_func(foladMerge)

    relpcap_feature = pd.concat([stock_percapita,relpcap_feature_data],axis=1)

    def power_count_features(relpcap_feature):
        df = relpcap_feature
        df.index = [pd.Timestamp(item) for item in df['date']]

        count_indices = np.where(df['indv_power']>=0)
        temp = np.zeros(len(df['indv_power']),)
        temp[count_indices,] = 1
        df['count_indices'] = temp
        count_5d = df['count_indices'].rolling(5).sum()
        count_30d = df['count_indices'].rolling(30).sum()

        return_df = pd.DataFrame({\
            'ind_power_count5d30' : count_5d/count_30d} )
        return_df.index = initial_index
        return return_df
    
    power_count_data = pcap_based_func(foladMerge)
  
    # concate
    features = pd.concat([pcap_feature_data, mrkt_pcap_feature_data, mrkt_rolling_data, relpcap_feature_data, power_count_data], \
        axis=1)

    return features

inv_noninv_features = indv_nonindv_based_feature(foladMerge, wght_data, prop_features)
inv_noninv_features

In [42]:
#indv_nonindv_based_feature(foladMerge, wght_data, prop_features)

individual and non-individual features


,Individual_buy_per_capita,Individual_sell_per_capita,indv_net_count_prp7d30,indv_net_count_prp7d120,indv_net_value,indv_relnet_value_d14,indv_relnet_value_d30,indv_relnet_value_d60,indv_relnet_value_d90,indv_relnet_value_d120,...,indv_net_count_prp7d30,indv_net_count_prp7d120,indv_net_value,indv_relnet_value_d14,indv_relnet_value_d30,indv_relnet_value_d60,indv_relnet_value_d90,indv_relnet_value_d120,indv_relnet_value_d300,indv_relnet5d10d_value_d10
2007-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-10,1.004540e+08,1.273784e+08,0.055987,NaN,9.201858e+09,0.492317,0.180685,-2.562182,NaN,NaN,...,0.055987,NaN,9.201858e+09,0.492317,0.180685,-2.562182,NaN,NaN,NaN,-0.039814
2022-09-11,1.007742e+08,1.185709e+08,0.052123,NaN,3.536046e+10,0.894820,0.468073,-2.204765,NaN,NaN,...,0.052123,NaN,3.536046e+10,0.894820,0.468073,-2.204765,NaN,NaN,NaN,-0.049545
2022-09-12,7.224169e+07,1.232760e+08,0.063468,NaN,1.232380e+11,1.193644,0.948495,-1.639279,NaN,NaN,...,0.063468,NaN,1.232380e+11,1.193644,0.948495,-1.639279,NaN,NaN,NaN,-0.052066
2022-09-13,1.223385e+08,1.536095e+08,0.061890,NaN,1.387471e+11,1.163789,1.710589,-1.373544,NaN,NaN,...,0.061890,NaN,1.387471e+11,1.163789,1.710589,-1.373544,NaN,NaN,NaN,-0.044056


In [21]:
def calender_feature(df):
    print('calendar based features')
    
    df = foladHist
    return_df = pd.DataFrame({\
        'jdate':df['date'].jalali.to_jalali(), \
        'year':df['jdate'].jalali.year, \
        'month':df['jdate'].jalali.month, \
        'quarter':df['jdate'].jalali.quarter, \
        'day':df['jdate'].jalali.day, \
        'weekday':df['jdate'].jalali.weekday, \
        })
    
    return return_df

clnd_features = calender_feature(foladMerge)
clnd_features

calendar based features


,jdate,year,month,quarter,day,weekday
0,1385-12-20 00:00:00,1385,12,4,20,1
1,1385-12-21 00:00:00,1385,12,4,21,2
2,1385-12-22 00:00:00,1385,12,4,22,3
3,1385-12-23 00:00:00,1385,12,4,23,4
4,1385-12-26 00:00:00,1385,12,4,26,0
...,...,...,...,...,...,...
3412,1401-06-19 00:00:00,1401,6,2,19,0
3413,1401-06-20 00:00:00,1401,6,2,20,1
3414,1401-06-21 00:00:00,1401,6,2,21,2
3415,1401-06-22 00:00:00,1401,6,2,22,3
